In [ ]:
# default_exp callback.mixup

# Mixup Callback

> API details.

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
#export
from typing import Optional
from functools import partial

import torch
from torch import tensor
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.beta import Beta
import torchvision

import pytorch_lightning as pl
from pytorch_lightning.core import LightningModule
from pytorch_lightning.metrics import functional as FM
from pytorch_lightning.callbacks.base import Callback
from pytorch_lightning.utilities import rank_zero_info, rank_zero_warn
from pytorch_lightning.utilities.exceptions import MisconfigurationException

from isic.utils import unsqueeze

C:\Users\bomco\miniconda3\envs\devtorch\lib\site-packages\pytorch_lightning\utilities\distributed.py:23: UserWarning: Unsupported `ReduceOp` for distributed computing.
  warnings.warn(*args, **kwargs)


In [ ]:
#export
class NoneReduce():
    "A context manager to evaluate `loss_func` with none reduce."
    def __init__(self, loss_func): self.loss_func,self.old_red = loss_func,None

    def __enter__(self):
        if hasattr(self.loss_func, 'reduction'):
            self.old_red = self.loss_func.reduction
            self.loss_func.reduction = 'none'
            return self.loss_func
        else: return partial(self.loss_func, reduction='none')

    def __exit__(self, type, value, traceback):
        if self.old_red is not None: self.loss_func.reduction = self.old_red


In [ ]:
#export
def reduce_loss(loss, reduction='mean'):
    return loss.mean() if reduction=='mean' else loss.sum() if reduction=='sum' else loss

In [ ]:
#export
class Mixup(Callback):
    r"""
    Mixup
    """
    def __init__(self, alpha=0.4):
        self.distrib = Beta(tensor(alpha), tensor(alpha))

    def on_fit_start(self, trainer, pl_module):
        assert hasattr(pl_module, 'loss_func'), 'Your LightningModule should have loss_func attribute as your loss function.'
        self.old_lf, pl_module.loss_func = pl_module.loss_func, self.loss_func
        self.pl_module = pl_module

    def on_train_batch_start(self, trainer, pl_module, batch, batch_idx, dataloader_idx):
        device = pl_module.device
        x, y = batch
        x, y = x.to(device), y.to(device)
        bs = y.size(0)

        # Produce "bs" probability for each sample
        lam = self.distrib.sample((bs,)).squeeze().to(device)

        # Get those probability that >0.5, so that the first img (in the nonshuffle batch) has bigger coeff
        # Which avoid duplication mixup
        lam = torch.stack([lam, 1-lam], 1)
        self.lam = lam.max(1)[0]

        # Permute the batch
        shuffle = torch.randperm(bs).to(device)
        x1, self.yb1 = x[shuffle], y[shuffle]
        nx_dims = len(x.size())
        x_new = torch.lerp(x1, x, weight=unsqueeze(self.lam, n=nx_dims-1))
        
#         grid = torchvision.utils.make_grid(x_new)
#         trainer.logger.experiment.add_image('mixup', grid)

        pl_module.enhanced_batch = (x_new, y)

    def on_train_end(self, trainer, pl_module):
        pl_module.loss_func = self.old_lf

    def loss_func(self, pred, yb):
        #TODO check if training
        if self.pl_module.testing: return self.old_lf(pred, yb)
        with NoneReduce(self.old_lf) as lf:
            loss = torch.lerp(lf(pred, self.yb1), lf(pred,yb), self.lam)
        return reduce_loss(loss, getattr(self.old_lf, 'reduction', 'mean'))

In [ ]:
#export
class MixupDict(Callback):
    def __init__(self, alpha=0.4):
        self.distrib = Beta(tensor(alpha), tensor(alpha))

    def on_fit_start(self, trainer, pl_module):
        assert hasattr(pl_module, 'loss_func'), 'Your LightningModule should have loss_func attribute as your loss function.'
        self.old_lf, pl_module.loss_func = pl_module.loss_func, self.loss_func
        self.pl_module = pl_module

    def on_train_batch_start(self, trainer, pl_module, batch, batch_idx, dataloader_idx):
        xb, yb = batch["img"], batch["label"]
        bs = yb.size(0)

        # Produce "bs" probability for each sample
        lam = self.distrib.sample((bs,)).squeeze()

        # Get those probability that >0.5, so that the first img (in the nonshuffle batch) has bigger coeff
        # Which avoid duplication mixup
        lam = torch.stack([lam, 1-lam], 1)
        self.lam = lam.max(1)[0]

        # Permute the batch
        shuffle = torch.randperm(bs)
        xb_1, self.yb_1 = xb[shuffle], yb[shuffle]
        nx_dims = len(xb.size())
        weight = unsqueeze(self.lam, n=nx_dims-1)
        x_new = torch.lerp(xb_1, xb, weight=weight)
        grid = torchvision.utils.make_grid(x_new)
        trainer.logger.experiment.add_image('mixup', grid)
        grid_g = torchvision.utils.make_grid(xb)
        trainer.logger.experiment.add_image('norm', grid_g)
        batch["img"] = x_new

    def on_train_end(self, trainer, pl_module):
        pl_module.loss_func = self.old_lf

    def loss_func(self, pred, yb):
        #TODO check if training
        if self.pl_module.testing: return self.old_lf(pred, yb)
        with NoneReduce(self.old_lf) as lf:
            self.yb_1 = self.yb_1.to(pred.device)
            self.lam = self.lam.to(pred.device)
            loss = torch.lerp(lf(pred, self.yb_1), lf(pred,yb), self.lam)
        return reduce_loss(loss, getattr(self.old_lf, 'reduction', 'mean'))